**PLEASE UPVOTE https://www.kaggle.com/code/hechtjp/h-m-eda-rule-base-by-customer-age**

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
import time
import os
import copy
import gc
import re
import random
import pickle
import cudf

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)

import seaborn as sns
sns.set()

from pandas.io.json import json_normalize
from pprint import pprint
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()
from collections import Counter
from datetime import datetime, timedelta

In [ ]:
DEBUG = False
PATH_INPUT = r'../input/h-and-m-personalized-fashion-recommendations/'

In [ ]:
COUNT = 1

ORDER = 0
N = 12

In [ ]:
def display_df(df, head=3):
    print(f'SHAPE: {df.shape}\n')
    display(df.head(head))

In [ ]:
def pre_increment(name, local={}):
    if name in local:
        local[name] += 1
        
        return local[name]
    
    globals()[name] += 1
    
    return globals()[name]

In [ ]:
def info_df(df, count, order):
    if count:
        try:
            name = [x for x in globals() if globals()[x] is df][0]
        except IndexError:
            name = ''
        
        order = pre_increment('order')   
        print('=' * 30)
        print(f'{order} INFO_DF {name}:\n')
        display_df(df)

In [ ]:
articles_df = cudf.read_csv(PATH_INPUT + 'articles.csv', 
                            usecols=['article_id', 
                                     'product_group_name', 
                                     'perceived_colour_master_name'])
display_df(articles_df)

In [ ]:
customers_df = cudf.read_csv(PATH_INPUT + 'customers.csv',
                             usecols=['customer_id', 'age'])
display_df(customers_df)

In [ ]:
customers_df = customers_df.to_pandas()
bin_list = [-1, 19, 29, 39, 49, 59, 69, 119]
customers_df['age_bins'] = pd.cut(customers_df['age'], bin_list)

display_df(customers_df)

In [ ]:
age_missing = customers_df[customers_df['age_bins'].isnull()].shape[0]
age_missing

In [ ]:
transactions_df = cudf.read_csv(PATH_INPUT + 'transactions_train.csv',
                                usecols=['t_dat', 'customer_id', 'article_id'],
                                dtype={'t_dat': 'string',
                                       'customer_id': 'string',
                                       'article_id': 'int32'})
transactions_df['t_dat'] = cudf.to_datetime(transactions_df['t_dat'])
transactions_df.set_index('t_dat', inplace=True)

display_df(transactions_df)

In [ ]:
recent_df = transactions_df.loc['2020-09-01':'2020-09-21']

display_df(recent_df)

In [ ]:
recent_df = recent_df.to_pandas()
recent_df = recent_df.merge(customers_df[['customer_id', 'age_bins']], on='customer_id', how='inner')

display_df(recent_df)

In [ ]:
recent_df = recent_df.groupby(['age_bins', 'article_id']).count().reset_index()\
    .rename(columns={'customer_id': 'counts'})

display_df(recent_df)

In [ ]:
bins_unique_list = recent_df['age_bins'].unique().tolist()
bins_unique_list

In [ ]:
bins_unique_list[0]

In [ ]:
ages_article_id = {}
count = COUNT
order = ORDER
for bins_unique in bins_unique_list:
    temp_df = recent_df[recent_df['age_bins'] == bins_unique]
    info_df(temp_df, count,order)
    
    temp_df = temp_df.sort_values(by='counts', ascending=False)
    info_df(temp_df, count, order)
    
    ages_article_id[bins_unique] = temp_df.head(100)['article_id'].values.tolist()
    
    count = 0

In [ ]:
from itertools import islice

for key, value in islice(ages_article_id.items(), 3):
    print(f'{key} {len(value)}')

In [ ]:
topcustcnt_byage_df = pd.DataFrame([ages_article_id])

display_df(topcustcnt_byage_df)

In [ ]:
topcustcnt_byage_df = pd.DataFrame([ages_article_id]).T.rename(columns={0: 'top_100'})

display_df(topcustcnt_byage_df)

In [ ]:
for i in topcustcnt_byage_df.index:
    topcustcnt_byage_df[i] = [len(set(topcustcnt_byage_df.at[i, 'top_100']) & \
                                  set(topcustcnt_byage_df.at[j, 'top_100'])) / 100 for j in topcustcnt_byage_df.index]
    
display_df(topcustcnt_byage_df)

In [ ]:
topcustcnt_byage_df = topcustcnt_byage_df.drop(columns='top_100')

display_df(topcustcnt_byage_df, head=10)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(topcustcnt_byage_df, cmap='winter', annot=True, cbar=False)

In [ ]:
bins_unique_list

In [ ]:
bins_unique_list = customers_df['age_bins'].unique().tolist()
bins_unique_list

In [ ]:
count = COUNT
order = ORDER
for bin_unique in bins_unique_list:
    
    df = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
                       usecols=['t_dat', 'customer_id', 'article_id'],
                       dtype={'t_dat': 'string',
                              'customer_id': 'string',
                              'article_id': 'int32'})
    info_df(df, count, order)
    
    if str(bin_unique) == 'nan':
        temp_customers_df = customers_df[customers_df['age_bins'].isnull()]
    else:
        temp_customers_df = customers_df[customers_df['age_bins'] == bin_unique]
        
    temp_customers_df = temp_customers_df.drop(['age_bins'], axis=1)
    temp_customers_df = cudf.from_pandas(temp_customers_df)
    info_df(temp_customers_df, count, order)
    
    df = df.merge(temp_customers_df[['customer_id', 'age']], on='customer_id', how='inner')
    info_df(df, count, order)
    
    print(f'TRANSACTION SHAPE FOR SCOPE {bin_unique}: {df.shape}\n')
    
    df['customer_id'] = df['customer_id'].str[-16:].str.hex_to_int().astype('int64')
    df['t_dat'] = cudf.to_datetime(df['t_dat'])
    
    last_date_train = df['t_dat'].max()
    
    if count:
        print('=' * 30 + f'\n4 LAST_DATE_TRAIN:\n{last_date_train}')
    
    tmp = df[['t_dat']].copy().to_pandas()
    tmp['weekday'] = tmp['t_dat'].dt.dayofweek
    info_df(tmp, count, order)
    
    tmp['t_dat_shiftday'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['weekday'] - 1, unit='D')
    info_df(tmp, count, order)
    
    tmp.loc[tmp['weekday'] >= 2, 't_dat_shiftday'] = tmp.loc[tmp['weekday'] >= 2, 't_dat_shiftday'] + \
        pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['weekday'] >= 2])) * 7, unit='D')
    info_df(tmp, count, order)
    
    df['t_dat_shiftday'] = tmp['t_dat_shiftday'].values
    info_df(df, count, order)
    
    weekly_sales = df.drop('customer_id', axis=1).groupby(['t_dat_shiftday', 'article_id']).count().reset_index()
    info_df(weekly_sales, count, order)
    
    weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
    info_df(weekly_sales, count, order)
    
    df = df.merge(weekly_sales, on=['t_dat_shiftday', 'article_id'], how='left')
    info_df(df, count, order)
    
    weekly_sales = weekly_sales.reset_index().set_index('article_id')
    info_df(weekly_sales, count, order)
    
    df = df.merge(weekly_sales.loc[weekly_sales['t_dat_shiftday'] == last_date_train, ['count']], 
                 on='article_id',
                 suffixes=('', '_targ'))
    info_df(df, count, order)
    
    df['count_targ'].fillna(0, inplace=True)
    del weekly_sales
    
    df['quotient'] = df['count_targ'] / df['count']
    info_df(df, count, order)
    
    target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
    info_df(target_sales, count, order)
    
    general_pred = target_sales.nlargest(N).index.to_pandas().tolist()
    general_pred = ['0' + str(article_id) for article_id in general_pred]
    general_pred_str = ' '.join(general_pred)
    
    del target_sales
    
    purchase_dict = {}
    tmp = df.copy().to_pandas()
    info_df(tmp, count, order)
    
    tmp['x'] = ((last_date_train - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
    info_df(tmp, count, order)
    
    tmp['dummy_1'] = 1
    tmp['x'] = tmp[['x', 'dummy_1']].max(axis=1)
    info_df(tmp, count, order)
    
    a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
    tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c * tmp['x']) - d
    info_df(tmp, count, order)
    
    tmp['dummy_0'] = 0
    tmp['y'] = tmp[['y', 'dummy_0']].max(axis=1)
    tmp['value'] = tmp['quotient'] * tmp['y']
    info_df(tmp, count, order)
    
    tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
    info_df(tmp, count, order)
    
    tmp = tmp.reset_index()
    
    tmp = tmp.loc[tmp['value'] > 0]
    info_df(tmp, count, order)
    
    tmp['rank'] = tmp.groupby('customer_id')['value'].rank('dense', ascending=False)
    info_df(tmp, count, order)
    
    tmp = tmp.loc[tmp['rank'] <= 12]
    info_df(tmp, count, order)
    
    purchase_df = tmp.sort_values(['customer_id', 'value'], ascending=False).reset_index(drop=True)
    info_df(purchase_df, count, order)
    
    purchase_df['prediction'] = '0' + purchase_df['article_id'].astype('str') + ' '
    info_df(purchase_df, count, order)
    
    purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
    info_df(purchase_df, count, order)
    
    purchase_df = cudf.DataFrame(purchase_df)
    
    sub = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv',
                        usecols=['customer_id'],
                        dtype={'customer_id': 'string'})
    info_df(sub, count, order)
    
    num_customers = sub.shape[0]
    sub = sub.merge(temp_customers_df[['customer_id', 'age']], on='customer_id', how='inner')
    info_df(sub, count, order)
    
    sub['customer_id_2'] = sub['customer_id'].str[-16:].str.hex_to_int().astype('int64')
    info_df(sub, count, order)
    
    sub = sub.merge(purchase_df,
                    left_on='customer_id_2',
                    right_on='customer_id',
                    how='left',
                    suffixes={'', '_ignored'})
    info_df(sub, count, order)
    
    sub = sub.to_pandas()
    sub['prediction'] = sub['prediction'].fillna(general_pred_str)
    sub['prediction'] = sub['prediction'] + ' ' + general_pred_str
    sub['prediction'] = sub['prediction'].str.strip()
    sub['prediction'] = sub['prediction'].str[:131]
    
    sub = sub[['customer_id', 'prediction']]
    sub.to_csv(f'submission_{str(bin_unique)}.csv', index=False)
    info_df(sub, count, order)
    
    print(f'SAVED PREDICTION FOR {bin_unique}, SHAPE: {sub.shape}\n')
    print('-' * 50)
    
    count = 0
    
print('FINISHED')
print('=' * 50)

In [ ]:
for i, bin_unique in enumerate(bins_unique_list):
    temp_df = cudf.read_csv(f'submission_{str(bin_unique)}.csv')
    if i == 0:
        sub_df = temp_df
    else:
        sub_df = cudf.concat([sub_df, temp_df], axis=0)
        
assert sub_df.shape[0] == num_customers,\
    f'SUB_DF ROWS NUMBER IS NOT CORRECT {sub_df.shape[0]} VS {num_customers}'
    
sub_df.to_csv('by_cust_age__.csv', index=False)

display_df(sub_df)

In [ ]:
check_df = cudf.read_csv('./by_cust_age__.csv')

display_df(check_df)